# Information extraction using RegEx

We have a messy medical data and using regex let's extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants that one might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once these date patterns have been extracted from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Also we need to watch out for potential typos as this is a raw, real-life derived dataset.

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [3]:
import warnings
warnings.filterwarnings('ignore')
def date_sorter():
    # Regex for different date formats
    reg1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    reg2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{1,2}[,]{0,1}[+\s]\d{4})'
    reg3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    reg4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    reg5 = '(\d{1,2}[/-][1|2]\d{3})'
    reg6 = '([1|2]\d{3})'
    complete_reg = '(%s|%s|%s|%s|%s|%s)' %(reg1, reg2, reg3, reg4, reg5, reg6)
    # Extracting date from the corpus.
    date = df.str.extract(complete_reg)
    # Handling typo
    date = date.iloc[:,0].str.replace('Janaury', 'January').str.replace('Decemeber', 'December')
    # Converting to date-time format
    date = pd.Series(pd.to_datetime(date))
    # Sorting
    date = date.sort_values(ascending=True).item
    return date
date_sorter()

<bound method IndexOpsMixin.item of 9     1971-04-10
84    1971-05-18
2     1971-07-08
53    1971-07-11
28    1971-09-12
474   1972-01-01
153   1972-01-13
13    1972-01-26
129   1972-05-06
98    1972-05-13
111   1972-06-10
225   1972-06-15
31    1972-07-20
171   1972-10-04
191   1972-11-30
486   1973-01-01
335   1973-02-01
415   1973-02-01
36    1973-02-14
405   1973-03-01
323   1973-03-01
422   1973-04-01
375   1973-06-01
380   1973-07-01
345   1973-10-01
57    1973-12-01
481   1974-01-01
436   1974-02-01
104   1974-02-24
299   1974-03-01
         ...    
220   2012-06-25
208   2012-09-01
243   2012-09-01
139   2012-10-21
320   2012-11-01
383   2012-12-01
244   2013-01-01
286   2013-01-01
480   2013-01-01
431   2013-04-01
279   2013-09-01
198   2013-10-11
381   2014-01-01
463   2014-01-01
366   2014-07-01
439   2014-10-01
255   2014-10-01
401   2014-12-01
475   2015-01-01
257   2015-09-01
152   2015-09-28
235   2015-10-01
464   2016-01-01
253   2016-02-01
427   2016-05-01
231   2016-0